In [1]:
import tensorflow as tf
import os

In [4]:
class MyFashionMnist(object):
  def train(self):

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.summary()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=10)

    model.evaluate(x_test,  y_test, verbose=2)

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        #DOCKER_REGISTRY = 'keti-docker.registry.local:5000' #'kubeflow-registry.default.svc.cluster.local:30000'        
        DOCKER_REGISTRY = '192.168.0.180:5000'        
        fairing.config.set_builder(
            'append',
            image_name='fairing-job',
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu', #'brightfly/kubeflow-jupyter-lab:tf2.0-gpu',
            registry=DOCKER_REGISTRY, 
            push=True)
        # cpu 2, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='space-openess', #'dudaji',
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=2,
                                                                       memory=5)]
         
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 220928 08:29:33 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f728056c2e8>
[I 220928 08:29:33 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f72e82e6828>
[I 220928 08:29:33 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f72e82e6828>
[W 220928 08:29:33 append:50] Building image using Append builder...
[I 220928 08:29:33 base:105] Creating docker context: /tmp/fairing_context_cgut963v
[I 220928 08:29:33 converted_notebook:127] Converting fashion-mnist-fairing.ipynb to fashion-mnist-fairing.py
[I 220928 08:29:33 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 220928 08:29:35 append:54] Image successfully built in 2.3115898575633764s.
[W 220928 08:29:35 append:94] Pushing image 192.168.0.180:5000/fairing-job:A3C1679...
[I 220928 08:29:35 docker_creds_:234] 

4423680/4422102 [==============================] - 0s 0us/step
2022-09-28 08:29:43.534807: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-09-28 08:29:43.560624: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2022-09-28 08:29:43.562330: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5fafa70 executing computations on platform Host. Devices:
2022-09-28 08:29:43.562384: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
_________________________________________________________________
dense (Dense)                (None, 128)               100480
___________________________

[W 220928 08:31:13 job:162] Cleaning up job fairing-job-fsdc4...
